# PM2.5 concentration spatial distribution estimation in China 
## Abstract  

China, as the largest developing country in the world, has been facing increasingly severe air quality problems accompanying the continuous advancement of industrialization and urbanization. According to the “2017 China Environmental Situation Bulletin” released by the Ministry of Ecology and Environment, among the 338 cities at the prefecture level and above, 239 cities have exceeded the national ambient air quality standards, with a proportion of more than 70%. Air quality problems have seriously affected people's daily travel and physical health, restricted sustainable economic development, and become a hot issue for public and government attention. PM2.5 is a respirable particle with a diameter of no more than 2.5 micrometers in the aerodynamic field, and is one of the main indicators for evaluating air quality. Comprehensive understanding of the spatial distribution of PM2.5 concentration, representing the spatial process and environmental behavior of atmospheric pollution, is of great significance and guidance value for supporting atmospheric pollution monitoring and early warning, comprehensive treatment, protecting human health and social sustainable development. By the end of 2017, the China National Environmental Monitoring Center had established over 400 ground-level air quality monitoring stations, and released hourly air quality monitoring data including PM2.5, providing high-precision and reliable real-time monitoring results. However, due to the uneven spatial distribution and low coverage of ground monitoring stations, it is difficult for existing studies to effectively analyze and deeply mine the spatiotemporal data of their monitoring data. Unlike ground monitoring, remote sensing observation based on satellites can obtain high-coverage atmospheric environment spatial datasets, such as atmospheric aerosol optical depth (AOD). Numerous studies have shown that there is a strong correlation between AOD and PM2.5 concentration. Research on PM2.5 concentration Spatial regression relationships with factors such as AOD retrieved from remote sensing inversion can provide effective solutions for obtaining the spatial distribution of PM2.5 concentration in the entire study area. Methodology Based on GWR geographical weighted regression thinking, Wu SenSen combines OLR with neural network model to propose a Geographically Neural Network Weighted Regression (GNNWR) model. By utilizing the learning ability of neural networks, this model can handle the spatial heterogeneity and complex nonlinear characteristics of regression relationships, which has better fitness accuracy and prediction performance compared to models such as OLR and GWR. The purpose of this case is to establish a PM2.5 concentration spatial estimation model based on GNNWR to achieve accurate fitting of spatial heterogeneity and nonlinear characteristics in PM2.5 regression relationships, and then obtain high-precision and high-reasonability PM2.5 concentration spatial distribution in China.  

## Data description  

Many studies have shown that integrating meteorological conditions such as temperature, precipitation, wind speed, wind direction, and surface elevation factors can further improve the accuracy of PM2.5 spatial estimation. In this case, in addition to selecting AOD data as an auxiliary factor, temperature (TEMP), precipitation (TP), wind speed (WS), wind direction (WD), and surface elevation (DEM) factors are added as input variables for the model. The research time scale is the average of 2017 year scale:
(1) PM2.5 monitoring site data. The hourly PM2.5 concentration observation values from January 1, 2017 to December 31, 2017 were obtained from the China Environmental Monitoring Station. PM2.5 concentrations were measured using cone-shaped element oscillation trace or beta attenuation methods following national standard GB3095-2012. PM2.5 data were averaged for one year's time scale
(2) Aerosol data. Aerosol data are obtained from the LAADS website including both Terra and Aqua dark pixel inversion products with a resolution of 3 km (MOD04_3K and MYD04_3k), as well as deep blue algorithm inversion products with a resolution of 10 km (MOD04_L2 and MYD04_L2). In this article, the 3 km resolution AOD products are the main data source for PM2.5 estimation. When there is a missing value in the 3 km resolution data, a resolution-matching product will be used as much as possible using a 10 km resolution product for resampling substitution to ensure the reliability of AOD data.
(3) DEM data. DEM data are obtained from the ETO-PO1 global surface elevation model of NOAA with a resolution of 1 arc minute
(4) Meteorological data including temperature, precipitation, wind speed, and wind direction are obtained from the ERA5 global climate reanalysis modele provided by ECMWF with hourly gridded data at a resolution of 0.5 degrees.   

## Model Introduction  

Based on the geographical weighted idea similar to GWR, the GNNWR model believes that the spatial heterogeneity of the regression relationship can be regarded as the varying levels of spatial nonstationarity at different locations that affect the "OLR regression relationship". Therefore, in the spatial estimation experiment of PM2.5 concentration in this case, the model structure of GNNWR is defined as follows:  
 
![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059648746_1.png)  

In this equation, (u_i, v_i) are the spatial coordinates of the i-th sample point, and β = (β_0, β_1, ..., β_6) are the regression coefficients of the OLR model, reflecting the average level of the PM2.5 regression relationship for the entire region. The estimation matrix of OLR coefficients is represented as follows:

![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059665465_2.png)  

of which: 

![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059673642_3.png)  


<br/><br/>  


![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694003342595_1.png)  
The spatial estimation model for PM2.5 concentration based on GNNWR

## Main Content  
1. Model Training 
2. Result Storage, Loading, and Visualization
3. Estimation 

# Part A：Preparation

## Import Necessary Packages

In [ ]:
from gnnwr import models,datasets
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap, MarkerCluster
from demo_utils import marker_map,Heatmap
import torch.nn as nn
from sklearn.metrics import r2_score as r2
import matplotlib.pyplot as plt

# Part B：Model Training

## Step 1：Import Training Data

In [ ]:
data = pd.read_csv(u'../data/pm25_data.csv')
data.head(5)

### View the distribution of training data

In [ ]:
from demo_utils import marker_map

In [ ]:
lon_center,lat_center = data['经度'].mean(),data['纬度'].mean()
locations = list(zip(data['纬度'],data['经度']))
markers = [{'location':locations,'color':'blue','desc':'data'}]
marker_map(markers,[lat_center,lon_center],4)


## Step 2：Partition Datasets

In [ ]:
train_dataset, val_dataset, test_dataset = datasets.init_dataset(data=data,
                                                        test_ratio=0.15,
                                                        valid_ratio=0.15,
                                                        x_column=['dem', 'w10','d10','t2m','aod_sat','tp'],
                                                        y_column=['PM2_5'],
                                                        spatial_column=['经度','纬度'],
                                                        sample_seed=23,
                                                        batch_size=64)

### Checking the partitioning of the datasets

In [ ]:
markers_1 = [{'location':list(zip(train_dataset.dataframe['纬度'],train_dataset.dataframe['经度'])),'color':'blue'},
             {'location':list(zip(val_dataset.dataframe['纬度'],val_dataset.dataframe['经度'])),'color':'green'},
             {'location':list(zip(test_dataset.dataframe['纬度'],test_dataset.dataframe['经度'])),'color':'red'}
            ]
marker_map(markers_1,[lat_center,lon_center],4)


## Step 3：Initialize GNNWR Model

In [ ]:
gnnwr = models.GNNWR(train_dataset = train_dataset,
                     valid_dataset = val_dataset, 
                     test_dataset = test_dataset,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adadelta",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result/gnnwr_logs",
                     write_path = "./demo_result/gnnwr_runs"
                     )

## Step 4：Model Training

In [ ]:
gnnwr.run(max_epoch = 200,print_frequency = 10)

## Step 5：模型评价与结果分析

### 输出模型训练结果

In [ ]:
gnnwr.result()


### Save the training result

In [ ]:
gnnwr.reg_result('./demo_result/GNNWR_PM25_Result.csv')

### Drawing the heat map for the distribution of weights of each variables 

In [ ]:
result_data = gnnwr.getWeights()


#### DEM

In [ ]:
weight_dem = [[row['纬度'],row['经度'],row['weight_dem']]for index,row in result_data.iterrows()]
Heatmap(weight_dem,[lat_center,lon_center])


#### AOD

In [ ]:
weight_aod = [[row['纬度'],row['经度'],row['weight_aod_sat']]for index,row in result_data.iterrows()]
Heatmap(weight_dem,[lat_center,lon_center])

### Precipitation

In [ ]:
weight_tp = [[row['纬度'],row['经度'],row['weight_tp']]for index,row in result_data.iterrows()]
Heatmap(weight_tp,[lat_center,lon_center])

#### Temperature

In [ ]:
weight_t2m = [[row['纬度'],row['经度'],row['weight_t2m']]for index,row in result_data.iterrows()]
Heatmap(weight_t2m,[lat_center,lon_center])

#### Wind Speed

In [ ]:
weight_w10 = [[row['纬度'],row['经度'],row['weight_w10']]for index,row in result_data.iterrows()]
Heatmap(weight_w10,[lat_center,lon_center])

#### Wind Direction

In [ ]:
weight_d10 = [[row['纬度'],row['经度'],row['weight_d10']]for index,row in result_data.iterrows()]
Heatmap(weight_d10,[lat_center,lon_center])

#### Bias

In [ ]:
weight_bias = [[row['纬度'],row['经度'],row['bias']]for index,row in result_data.iterrows()]
Heatmap(weight_bias,[lat_center,lon_center])

# Part C：Saving and Loading

## Step 1：Saving Datasets

In [ ]:
# make sure dir is not exist
train_dataset.save('./demo_result/gnnwr_datasets/train_dataset')
val_dataset.save('./demo_result/gnnwr_datasets/val_dataset')
test_dataset.save('./demo_result/gnnwr_datasets/test_dataset')

## Step 2：Loading Datasets

In [ ]:
train_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/train_dataset/')
val_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/val_dataset/')
test_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/test_dataset/')

## Step 3：Loading Model

### Initialize GNNWR Model

In [ ]:
gnnwr_load = models.GNNWR(train_dataset = train_dataset_load,
                     valid_dataset = val_dataset_load, 
                     test_dataset = test_dataset_load,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adadelta",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result/gnnwr_logs",
                     write_path = "./demo_result/gnnwr_runs"
                     )

### Loading Parameters

In [ ]:
gnnwr_load.load_model('./demo_result/gnnwr_models/GNNWR_PM25.pkl')
gnnwr_load.result()

# Part D：Estimation

## Step 1：Import Estimation Data

In [ ]:
pred_data = pd.read_csv(u'../data/pm25_predict_data.csv')

## Step 2：Initialize Estimation Dataset

In [ ]:
pred_dataset = datasets.init_predict_dataset(data = pred_data,train_dataset = train_dataset,x_column=['dem', 'w10','d10','t2m','aod_sat','tp'],spatial_column=['经度','纬度'])

## Step 3：Estimate

In [ ]:
res = gnnwr.predict(pred_dataset)
res.head(5)

In [ ]:
hmap_pred = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_pred = [[row['纬度'],row['经度'],row['pred_result']]for index,row in res.iterrows()]
hmap_pred.add_child(HeatMap(weight_pred,gradient={.3:"blue",.6:"green",.9:"yellow",1:"red"}))
hmap_pred